# Uncertainty estimation prototype

For https://github.com/gammapy/gammapy/pull/2255

Quick prototype how to compute spectral model error band using

1. Differentials
2. Samples

This is not clean code or a good solution yet, but it shows that we can compute this ourselves with ~ 30 lines of code in the spectral model base class.

We use a spectral model with the values and covariance taken from here:
https://github.com/open-gamma-ray-astro/joint-crab/blob/master/results/fit/fit_veritas.yaml

In [1]:
import numpy as np
import astropy.units as u
from gammapy.modeling.models import LogParabolaSpectralModel
from scipy.optimize.slsqp import approx_jacobian

In [2]:
model = LogParabolaSpectralModel(
    amplitude=3.76e-11 * u.Unit("cm-2 s-1 TeV-1"),
    reference=1 * u.TeV,
    alpha=2.44,
    beta=0.25,
)
model.parameters.covariance = [
    [1.31e-23, 0, -6.80e-14, 3.04e-13],
    [0, 0, 0, 0],
    [-6.80e-14, 0, 0.00899, 0.00904],
    [3.04e-13, 0, 0.00904, 0.0284],
]

## Differentials

The current `evaluate_error` is based on the Python `uncertainties` package,
which is based on differentials (computed with high precision, using autograd).

Unfortunately it doesn't work for all models, and doesn't support `astropy.units.Quantity`,
so let's code up our own solution. It will be less accurate (using finite differential steps)
and probably also slower, but it will work for any model.

To do the error propagation, we use the common approximation:
https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Non-linear_combinations

- `p` = parameter vector
- `C` = covariance matrix for parameters `p`
- `f` = derived quantity, here `dN/dE(E)`
-  `df/dp` = partial derivative vector, how `f` changes with `p`

``f_err ^ 2 = (df/dp) @ C @ (df/dp)``

In [3]:
def gradient_step(model, energy, eps=1e-6):
    n = len(model.parameters)
    f = model(energy)
    shape = (n, len(np.atleast_1d(energy)))
    df_dp = np.zeros(shape)
    
    for idx, parameter in enumerate(model.parameters):
        if parameter.frozen:
            continue

        # TODO: is this a good step? Is there a better way?        
        dp = eps * model.parameters.error(idx)
        parameter.value += dp
        df = model(energy) - f
        df_dp[idx] = df.value / dp

        # Reset model to original parameter
        parameter.value -= dp
    
    return df_dp    


def gradient_scipy(model, energy, eps=1e-12):
    x = model.parameters.values
    frozen = np.array([_.frozen for _ in model.parameters])
    
    def func(xk):
        return model.evaluate(energy.to_value("TeV"), *xk)

    grad = approx_jacobian(x=x, func=func, epsilon=eps)
    grad[:, frozen] = 0
    return grad.T

In [4]:
%%timeit
gradient_step(model, [1, 10, 100] * u.TeV)

996 µs ± 32.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%%timeit
gradient_scipy(model, [1, 10, 100] * u.TeV)

327 µs ± 3.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
def evaluate_error(model, energy):
    """New implementation of model.evaluate_error."""
    C = model.parameters.covariance
    df_dp = gradient_scipy(model, energy)
    f_cov = np.dot(df_dp.T, np.dot(C, df_dp))
    err = np.sqrt(np.diagonal(f_cov))
    return err

In [7]:
model.evaluate_error(1 * u.TeV)[1].value

3.6193922141707712e-12

In [8]:
evaluate_error(model, 1 * u.TeV)

array([3.61939221e-12])

In [9]:
model.evaluate_error([0.1, 1, 10, 100] * u.TeV)[1].value

array([2.02268909e-09, 3.61939221e-12, 3.62436537e-14, 9.37507775e-18])

In [10]:
evaluate_error(model, [0.1, 1, 10, 100] * u.TeV)

array([2.02262782e-09, 3.61939221e-12, 3.62442853e-14, 9.37521522e-18])

## Flux

Besides differential flux `dnde` computed above, we also need to compute integral flux.
We could try to generalise the error propagation code somehow, and to avoid code duplication.
Here's one example how this might be achieved, by having the quantity of interest as a function ``fct`` and passing that to a generic error propatation routine.

In [11]:
def propagate_error(fct, parameters, eps=0.01):
    n = len(parameters)
    C = parameters.covariance
    f = fct()
    df_dp = np.zeros(n)
    
    for idx, parameter in enumerate(parameters):
        if parameter.frozen:
            continue

        # TODO: is this a good step? Is there a better way?        
        dp = eps * parameters.error(idx)
        parameter.value += dp
        df = fct() - f
        df_dp[idx] = df.value / dp
    
        # Reset model to original parameter
        parameter.value -= dp
    
    f_cov = df_dp @ C @ df_dp
    return np.sqrt(f_cov)

def evaluate_error(model, energy, eps=0.01):
    def dnde():
        return model(energy)

    return propagate_error(dnde, model.parameters, eps)

def integral_error(model, emin, emax, eps=0.01):
    def integral():
        return model.integral(emin, emax)

    return propagate_error(integral, model.parameters, eps)

## Samples

Sample parameters from the covariance matrix,
and compute the distribution of the quantity of interest.

- https://docs.scipy.org/doc/numpy/reference/random/generated/numpy.random.Generator.multivariate_normal.html
- https://docs.astropy.org/en/stable/uncertainty/

This is easy to implement if we do a Python for loop over sampled parameter sets.
Typically one would do 10_000 to get a ~ 1% error.
That's probably pretty slow.

TODO: can we vectorise the evaluation over array energies and array parameters?

In [12]:
def evaluate_error_sample(model, energy, err_scale=1, n_samples=1000):
    """Error propagation using sampling.
    
    Normally `err_scale=1` gives accurate results.
    A small value like `err_scale=0.01` allows to reproduce
    the method and results from the differential error propagation
    method. 
    
    TODO: document
    """
    parameters = model.parameters
    cov = (err_scale ** 2) * parameters.covariance
    samples = np.random.multivariate_normal(
        parameters.values, cov, size=n_samples
    )
    f_samples = np.zeros(n_samples)
    for idx, sample in enumerate(samples):
        with parameters.restore_values:
            # TODO: should add Numpy array-based setter on Parameters
            for par, val in zip(parameters, sample):
                par.value = val

            f_samples[idx] = model(energy).value
    
    return {
        "samples": f_samples,
        "mean": np.mean(f_samples),
        "error": np.std(f_samples) / err_scale,
    }

In [13]:
%%time
dnde = evaluate_error_sample(model, 1 * u.TeV, err_scale=0.01, n_samples=10_000)
# plt.hist(dnde["samples"], bins=100)
print(dnde["mean"], dnde["error"])

3.759983711667153e-11 3.0066520509718442e-12
CPU times: user 2.5 s, sys: 20.2 ms, total: 2.52 s
Wall time: 2.43 s


In [14]:
model.evaluate_error(1 * u.TeV).value

array([3.76000000e-11, 3.61939221e-12])

The mean is correct, but the standard error from the MCMC doesn't match the value from `model.evaluate_error`, for unknown reasons. If you know why, please leave a comment in r https://github.com/gammapy/gammapy/pull/2255 !